# Welcome to the EMA Backtest & Plotting Notebook



In [1]:
# Imports
#from math import pi
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import Range1d, LinearAxis, HoverTool
from bokeh.models.glyphs import Step
from bokeh.layouts import column
from datetime import datetime
from backfire import ema

In [2]:
# Main graphing function
def graph_candlestick(my_title, candle_df, fills, balance, settings, results, candle_resolution, dd_df):
    
    # Prep
    if candle_resolution == 'day':
        candle_df = candle_df.groupby(candle_df['time'].dt.date).agg({
            'open': 'first',  'high': max,
            'low': min, 'close': 'last',
            }).reset_index()
    
    candle_df = candle_df[:10080]
    
    fills = fills[fills.time <= candle_df.time.max()]
    
    buys = fills[fills['side']=='buy']
    sells = fills[fills['side']=='sell']
    inc_df = candle_df[candle_df.close > candle_df.open]
    dec_df = candle_df[candle_df.open > candle_df.close]
    #if results is not None:
        #my_title = f"""Final Balance: {round(results['final_bal'],2)},
        #ROI: {round(results['roi'],2)}, Hodl ROI: {round(results['hodl_roi'],2)}"""
    #else:
    #    my_title = "BTC Minute Data"
        
    # Width of candles... doesn't always work like it should
    #w = 18*60*60*1000 # candle period in ms
    res_len_secs = round((candle_df.time[2] - candle_df.time[1]).total_seconds())
    w = res_len_secs * 1000 # candle period in ms
    candle_tooltips = [
        ("date", "@time{%F}"),
        ("open", "@open{$0,0.00}"),
        ("low", "@low{$0,0.00}"),
        ("high", "@high{$0,0.00}"),
        ("close", "@close{$0,0.00}"),
        ]
    date_formatters={
            'time': 'datetime',
        }
    
    fill_tips = [
        ("date", "@time{%F}"),
        ("side", "@side"),
        ("price", "@price{$0,0.00}"),
        ("btc_val", "@btc_val{0,0.000}"),
        ("usd_val", "@usd_val{$0,0.00}"),
        ]
    
    hover_tools = "crosshair,wheel_zoom,pan,box_zoom,reset,save"
    
    inc_source = ColumnDataSource(inc_df)
    dec_source = ColumnDataSource(dec_df)
    buy_source = ColumnDataSource(buys)
    sell_source = ColumnDataSource(sells)

    
    # Initialize Plot
    p = figure(
        x_axis_type = "datetime",
        tools=[hover_tools],
        toolbar_location="below",
        toolbar_sticky=False,
        active_scroll="wheel_zoom",
        plot_width = 900,
        title = my_title,
        y_range = (candle_df['close'].min()*.9, candle_df['close'].max() * 1.1),
        logo = None)
    p.grid.grid_line_alpha=0.3
    
    # Add Candles
    # Wicks
    #p.segment(candle_df.time, candle_df.high,
    #          candle_df.time, candle_df.low,
    #          color="black", alpha = 0.4)
    # just draw price line
    p.line(candle_df.time, candle_df.close, color="red", alpha=0.5)

    
    # Bodies
    #green_bar = p.vbar('time', w,
    #       'open', 'close',
    #       fill_color = "#D5E1DD", line_color = "green",
    #       alpha = 0.5,
    #      source = inc_source)
    #red_bar = p.vbar('time', w,
    #       'open', 'close',
    #       fill_color = "#F2583E", line_color = "red",
    #       alpha = 0.5,
    #      source = dec_source)
    #p.add_tools(HoverTool(renderers=[green_bar,red_bar], tooltips=candle_tooltips, formatters = date_formatters))
    
    # Add Buy / Sell Indicators
    buy_circle = p.circle('time', 'price', color = "green", size = 8, alpha = 0.5, source = buy_source)
    sell_circle = p.circle('time', 'price', color = "red", size = 8, alpha = 0.5, source = sell_source)
    p.add_tools(HoverTool(renderers=[buy_circle, sell_circle], tooltips=fill_tips, formatters = date_formatters))
    
    # Add Running Balance Line
    #p.extra_y_ranges = {"running_bal": Range1d(start=fills.bal_usd.min(), end=fills.running_bal.max())}
    p.extra_y_ranges = {"running_bal": Range1d(start=0, end=fills.running_bal.max())}
    p.line(fills.time, fills.running_bal, color="deepskyblue", y_range_name="running_bal", alpha = 0.5)
    
    p.line(balance.time, balance.running_bal, line_dash="4 4", color="deepskyblue", y_range_name="running_bal", alpha = 0.4)
    
    p.line(balance.time, balance.drawdowns, color="slategray", y_range_name="running_bal", alpha = 0.5)
    
    
    #p.line(fills.time, fills.p_usd, color="deepskyblue", y_range_name="running_bal", alpha = 0.3)
    #p.step(fills.time, fills.bal_usd, color='purple', y_range_name='running_bal', alpha = 0.3, mode='after')
    p.add_layout(LinearAxis(y_range_name="running_bal"), 'right',)
    
    # Add EMA lines
    #if res_len_secs == 60:
    #    p.line(candle_df.time, candle_df.upper_ema, color="purple", alpha = 0.02)
    #    p.line(candle_df.time, candle_df.lower_ema, color="purple", alpha = 0.02)
    #    band_x = np.append(candle_df.time, candle_df.time[::-1])
    #    band_y = np.append(candle_df.lower_ema, candle_df.upper_ema[::-1])
    #    p.patch(band_x, band_y, color='purple', fill_alpha=0.03)
    
    
    # add the settings used and timeframe at the bottom:
    p.xaxis.axis_label = settings
    p.xaxis.axis_label_text_color = "#aa6666"
    #p.xaxis.axis_label_standoff = 30
    
    return(p)

## Backtest: Load data and set variables

In [3]:
# Load Data

start_time = "2018-05-01"
end_time = "2018-05-22"

# Note csv is updated daily around 00:30AM UTC (8:30AM China) for the previous days data
minute_df = pd.read_csv('~/backfire/data/resources/coinbase_fixed_2017-01-01_current.csv', 
                           usecols=['time', 'open', 'low', 'high', 'close'])
day_df, minute_df = ema.prep_data(minute_df, start_time, end_time)

In [4]:
# Variables

bt_vars = ema.BacktestSettings()

bt_vars.set_upper_window(64)
bt_vars.set_lower_window(64)

upper_factor = 1.0128
lower_factor = 0.9744
bt_vars.set_factor_high(upper_factor)
bt_vars.set_factor_low(lower_factor)
bt_vars.set_buy_pct_usd(0.5)
bt_vars.set_sell_pct_btc(0.12)

bt_vars.set_min_usd(100)
bt_vars.set_min_btc(.001)
bt_vars.set_principle_usd(35000)
bt_vars.set_principle_btc(.05)

In [ ]:
1-.0256

## Backtest: Running the logic

`ema.single_backtest` runs a single instance of a backtest and returns three pieces of information:

1. *minute_df* has columns added for signals which can be used for plotting
2. *result_overview* is a dictionary with some basic overview stats for the test. Note this is the same as a single record in the multi backtests
3. *fills_df* is a dataframe of each fill (buy / sell) made during backtest

The backtest currently is working well and can be run for any time period of data.

In [5]:
# Run backtest
minute_df, result_overview, fills_df = ema.single_backtest(minute_df, bt_vars)

In [6]:
# View Results
pd.DataFrame([result_overview])[['sd', 'ed', 'upper_factor', 
                                 'lower_factor', 'upper_window',
                                 'lower_window', 'hodl_roi', 'roi', 'usd_bal', 'btc_bal']]

,sd,ed,upper_factor,lower_factor,upper_window,lower_window,hodl_roi,roi,usd_bal,btc_bal
0,2018-05-01,2018-05-22,1.0128,0.9744,64,64,-0.091554,-0.066001,28626.387586,0.484135


## Backtest: Plotting results with Bokeh

`graph_candlestick()` is the only graphing function currently. It builds a plot called p which is then displayed with `show(p)`


* We can't graph more than a few days of minute_df it doesn't display well and can also cause the notebook to stall
* To plot longer periods of time use day_df ie, `candle_df = day_df`
    * Note TODO: graphing day_df is still slow, probably from the ema lines being minute data still
* Bokeh is fairly straight forward, feel free to play with the graph above to add lines you'd like


In [7]:

# merge candle_df with fills_df for further calculation

fm = pd.merge(minute_df, fills_df, left_on='time', right_on='time', how='outer').fillna(0)

# recalculate running balance for every entry
#fm['bal_btc'] = fm['bal_btc'] + bt_vars.principle_btc
#fm['bal_usd'] = fm['bal_usd'] + bt_vars.principle_usd
#fm['running_bal'] = (fm['bal_btc'] * fm['price']) + fm['bal_usd']


fm2 = minute_df
fm2 = fm.groupby(fm['time'].dt.date).agg({
    'open': 'first', 
    'high': 'max',
    'low': 'min', 
    'close': 'last',
    'btc_val': 'sum', # amount of btc exchanged this day
    'usd_val': 'sum' # amount of usd exchanged this day
    }).reset_index()
    
fm2 = fm2[:10080]

#fm2

fm2['bal_btc'] = fm2.btc_val.cumsum() + bt_vars.principle_btc
fm2['bal_usd'] = fm2.usd_val.cumsum() + bt_vars.principle_usd
fm2['bal_btc_as_usd'] =  (fm2['bal_btc'] * fm2['close'])
fm2['running_bal'] =  (fm2['bal_btc'] * fm2['close']) + fm2['bal_usd']
#fm2['usd_change'] =  (fm2['bal_btc'] * fm2['close']) + fm2['bal_usd']

#fm2

In [8]:
# try calculate max draw down"

def create_drawdowns(equity_curve):
    """
    Calculate the largest peak-to-trough drawdown of the PnL curve
    as well as the duration of the drawdown. Requires that the 
    pnl_returns is a pandas Series.

    Parameters:
    pnl - A pandas Series representing period percentage returns.

    Returns:
    drawdown, duration - Highest peak-to-trough drawdown and duration.
    """

    # Calculate the cumulative returns curve 
    # and set up the High Water Mark
    # Then create the drawdown and duration series
    hwm = [0]
    eq_idx = equity_curve.index
    drawdown = pd.Series(index = eq_idx)
    duration = pd.Series(index = eq_idx)
    df = pd.DataFrame(drawdown)

    # Loop over the index range
    for t in range(1, len(eq_idx)):
        cur_hwm = max(hwm[t-1], equity_curve[t])
        hwm.append(cur_hwm)
        drawdown[t]= hwm[t] - equity_curve[t]
        duration[t]= 0 if drawdown[t] == 0 else duration[t-1] + 1
    
    return drawdown.max(), duration.max(), df


max_dd, dd_duration, df_dd = create_drawdowns(fm2['running_bal'])
max_return = fm2['running_bal'].max()
dd_pct = max_dd / max_return

#print(max_dd, " duration", dd_duration)
#print(round (dd_pct * 100, 2),"%", max_return, max_dd )

#len(df_dd)
df_dd.fillna(0)


fm2['drawdowns'] = df_dd

#fm2

In [9]:
# minute_df limited to first 10080 minutes / 7 days

candle_resolution = 'day'

#make the title
title = "BTC Minute Data"
if result_overview is not None:
        title = f"""Final Balance: {round(result_overview['final_bal'],2)}, ROI: {round(result_overview['roi'],2)}, Hodl ROI: {round(result_overview['hodl_roi'],2)}
        Max Drawdown: {round(max_dd,2)} ({round (dd_pct * 100, 2)}%)   DD_Duration: {round(dd_duration,2)} """
        
# make string of the settings
s =  f"""TIME {start_time} to {end_time}      SETTINGS   {bt_vars.upper_window}   {bt_vars.lower_window}   {bt_vars.factor_high}   {bt_vars.factor_low}   {bt_vars.buy_pct_usd}   {bt_vars.sell_pct_btc}"""

p = graph_candlestick(my_title = title,
                      candle_df = minute_df, 
                      fills = fills_df,
                      balance = fm2,
                      settings = s,
                      results = result_overview,
                      candle_resolution = candle_resolution,
                      dd_df = df_dd
                     )
output_notebook()
show(p)

Loading BokehJS ...

In [10]:
#  function to get running balance for any input price
#def get_running_balance(price)

# Main graphing function
def graph_balances(balance, settings, results):
    

    # Initialize Plot
    p = figure(
        x_axis_type = "datetime",
        toolbar_location="below",
        toolbar_sticky=False,
        active_scroll="wheel_zoom",
        plot_width = 900,
        title = "Balances chart",
        y_range = (balance['close'].min()*.9, balance['close'].max() * 1.1),
        logo = None)
    p.grid.grid_line_alpha=0.3
    
    
    p.line(balance.time, balance.close, color="red", alpha=0.5)

    p.extra_y_ranges = {"running_bal": Range1d(start=0, end=balance.running_bal.max())}

    #p.line(balance.time, (balance.running_bal + balance.usd_val), color="blue", y_range_name="running_bal", alpha = 0.5)
    
    p.line(balance.time, balance.bal_btc_as_usd, color="green", y_range_name="running_bal", alpha = 1)
    
    p.line(balance.time, balance.bal_usd, color="gray", y_range_name="running_bal", alpha = 0.5)
    
    #p.extra_y_ranges = {"bal_usd": Range1d(start=0, end=balance.bal_usd.max())}
    #p.line(balance.time, balance.bal_btc_as_usd, color="green", y_range_name="bal_usd", alpha = 0.5)
    
    #p.line(balance.time, balance.bal_usd, line_dash="4 4", color="blue", y_range_name="bal_usd", alpha = 0.6)
   



    p.add_layout(LinearAxis(y_range_name="running_bal"), 'right',)
    # add the settings used and timeframe at the bottom:
    p.xaxis.axis_label = settings
    p.xaxis.axis_label_text_color = "#aa6666"
    
    return(p)


b = graph_balances(balance = fm2, settings = s, results = result_overview )
output_notebook()
show(b)

Loading BokehJS ...

In [ ]:
#fills_df
fm2

In [ ]:
fills_df